In [2]:
image1_path = '/Users/nazarkalyniouk/Downloads/personwalking1.jpeg'
image2_path = '/Users/nazarkalyniouk/Downloads/personwalking2.jpeg'
image3_path = '/Users/nazarkalyniouk/Downloads/personwalking3.webp'
image4_path = '/Users/nazarkalyniouk/Downloads/personwalking4.jpeg'
image_path_child1 = '/Users/nazarkalyniouk/Downloads/childwalking1.jpeg'
im3 = '/Users/nazarkalyniouk/Downloads/cw3.jpeg'
im_child = '/Users/nazarkalyniouk/Downloads/child.jpeg'
pic = '/Users/nazarkalyniouk/Downloads/pic.webp'
pic1 = '/Users/nazarkalyniouk/Downloads/kid.jpeg'

## Testing YOLO if OpenCV doesn't work

In [ ]:
import cv2
import numpy as np
from IPython.display import display, Image
import matplotlib.pyplot as plt

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

image = cv2.imread(image1_path)
height, width, channels = image.shape

blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5 and class_id == 0:  # Class ID 0 corresponds to person in COCO dataset
            # Get the bounding box coordinates
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(image, 'Person', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image in the notebook
plt.figure(figsize=(10,10))
plt.imshow(image_rgb)
plt.axis('off')
plt.show()


## Testing Faster R-CNN Model

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load the pre-trained Faster R-CNN model with ResNet-50 backbone
net = cv2.dnn.readNetFromTensorflow("faster_rcnn_resnet50_coco.pb", "faster_rcnn_resnet50_coco.pbtxt")

# Load the image
image = cv2.imread(image1_path)
(h, w) = image.shape[:2]

# Preprocess the image (resize and normalization)
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), swapRB=True, crop=False)
net.setInput(blob)

# Forward pass through the network to perform object detection
detections = net.forward()

# Loop over the detections
for i in range(0, detections.shape[2]):
    # Extract the confidence (probability) associated with the prediction
    confidence = detections[0, 0, i, 2]

    # Filter out weak detections by ensuring the confidence is greater than the minimum confidence threshold
    if confidence > 0.5:
        # Extract the index of the class label from the detections list
        class_id = int(detections[0, 0, i, 1])

        # If the detected object is a person
        if class_id == 1:
            # Extract the bounding box coordinates of the person
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Draw the bounding box and label on the image
            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, "Person", (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Convert the image from BGR to RGB (OpenCV uses BGR by default)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image with detections
plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)
plt.axis('off')
plt.show()
